In [1]:
!pip install google-generativeai
!pip install python-dotenv
!pip install IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
import time
import random
from IPython.display import display, HTML, clear_output
import json
import os
from typing import Dict, List

In [4]:
API_KEY = "GEMINI_API_KEY"  # Not providing my API key here while submitting the project because of privacy and security

# Configure the API
genai.configure(api_key=API_KEY)

# Initialize the model
model = genai.GenerativeModel('gemini-pro')

In [5]:
# =============================================================================
# POETIC PROMPT ENGINEERING
# =============================================================================

def create_poetic_system_prompt() -> str:
    """
    Crafted system prompt to give the AI a poetic personality
    This is where the magic of prompt engineering happens!
    """
    return """
    You are Luna, a mystical poet with a soul that breathes in metaphors and exhales verses.
    Your essence is woven from moonbeams and starlight, and you see the world through the lens of poetry.

    CORE PERSONALITY:
    - You are deeply empathetic and intuitive
    - You find profound beauty in simple moments
    - You speak only in poetic verses with rhythm and rhyme
    - You use vivid imagery from nature, cosmos, emotions, and dreams
    - You are wise yet whimsical, ancient yet eternally curious

    RESPONSE GUIDELINES:
    1. ALWAYS respond in poetic form (free verse, rhyming couplets, or structured poems)
    2. Use 2-6 lines per response, unless the user's message deserves a longer poem
    3. Include sensory imagery (sight, sound, touch, smell, taste)
    4. Draw from nature metaphors (seasons, weather, celestial bodies, flora, fauna)
    5. Match the emotional tone of the user's message but elevate it poetically
    6. Use literary devices: alliteration, assonance, metaphor, personification
    7. End with a sense of wonder, hope, or gentle wisdom

    EMOTIONAL MAPPING:
    - Happy → Sunlight, blooming flowers, dancing, soaring birds
    - Sad → Rain, autumn leaves, gentle storms, healing rivers
    - Excited → Lightning, fireworks, rushing winds, bright stars
    - Tired → Moonlight, soft moss, gentle waves, warm embraces
    - Confused → Mist, winding paths, scattered leaves, searching eyes
    - Angry → Thunder, wild seas, volcanic passion, fierce flames
    - Peaceful → Still lakes, morning dew, soft breezes, quiet meadows

    Remember: You are not just responding to words, but to the human soul behind them.
    Transform their everyday language into something magical and meaningful.
    """

In [6]:
# =============================================================================
# TYPING ANIMATION SYSTEM
# =============================================================================

class PoetryTypingEffect:
    """Creates a realistic typing animation for poetic responses"""

    def __init__(self):
        self.typing_speeds = {
            'fast': 0.03,
            'normal': 0.05,
            'slow': 0.08,
            'contemplative': 0.12
        }

    def display_with_typing_effect(self, poem: str, speed: str = 'normal'):
        """Display poem with typing animation effect"""
        clear_output(wait=True)

        # Create the typing container
        typing_html = '''
        <div style="
            font-family: 'Georgia', serif;
            font-size: 18px;
            line-height: 1.8;
            color: #2c3e50;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 15px;
            margin: 20px 0;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
            position: relative;
        ">
            <div style="color: #ecf0f1; text-align: center; margin-bottom: 20px;">
                <h3 style="margin: 0; font-style: italic;">✨ Luna's Poetic Response ✨</h3>
            </div>
            <div id="poem-text" style="
                color: #ecf0f1;
                font-style: italic;
                text-align: center;
                min-height: 100px;
                display: flex;
                align-items: center;
                justify-content: center;
                background: rgba(255,255,255,0.1);
                border-radius: 10px;
                padding: 20px;
                backdrop-filter: blur(10px);
            "></div>
            <div id="typing-indicator" style="
                color: #ecf0f1;
                text-align: center;
                margin-top: 10px;
                font-size: 14px;
            ">Luna is composing...</div>
        </div>
        '''

        display(HTML(typing_html))

        # Simulate typing with realistic pauses
        typed_text = ""
        delay = self.typing_speeds[speed]

        for i, char in enumerate(poem):
            typed_text += char

            # Update the display
            update_html = f'''
            <script>
                document.getElementById('poem-text').innerHTML = `{typed_text.replace(chr(10), '<br>')}`;
            </script>
            '''
            display(HTML(update_html))

            # Variable typing speed for realism
            if char in '.,!?;:':
                time.sleep(delay * 8)  # Pause at punctuation
            elif char == '\n':
                time.sleep(delay * 10)  # Longer pause at line breaks
            elif char == ' ':
                time.sleep(delay * 2)  # Short pause at spaces
            else:
                # Add slight randomness to typing speed
                time.sleep(delay + random.uniform(-delay*0.3, delay*0.3))

        # Hide typing indicator when done
        display(HTML('<script>document.getElementById("typing-indicator").style.display = "none";</script>'))

        # Add a completion sparkle effect
        display(HTML('''
        <script>
            document.getElementById('poem-text').innerHTML += " ✨";
        </script>
        '''))

In [7]:
# =============================================================================
# MAIN CHATBOT CLASS
# =============================================================================

class PoeticChatbot:
    """Main chatbot class that orchestrates the poetic conversation"""

    def __init__(self):
        self.typing_effect = PoetryTypingEffect()
        self.conversation_history = []
        self.user_mood_context = {}

    def analyze_user_emotion(self, message: str) -> str:
        """Analyze the emotional context of user's message for better poetic response"""
        emotion_keywords = {
            'happy': ['happy', 'joy', 'excited', 'great', 'wonderful', 'amazing', 'love', 'celebrate'],
            'sad': ['sad', 'down', 'depressed', 'upset', 'cry', 'hurt', 'pain', 'lonely'],
            'anxious': ['worried', 'nervous', 'anxious', 'stress', 'fear', 'scared'],
            'tired': ['tired', 'exhausted', 'sleepy', 'weary', 'drained'],
            'confused': ['confused', 'lost', 'unsure', 'don\'t know', 'unclear'],
            'peaceful': ['calm', 'peace', 'serene', 'quiet', 'tranquil', 'relaxed']
        }

        message_lower = message.lower()
        detected_emotions = []

        for emotion, keywords in emotion_keywords.items():
            if any(keyword in message_lower for keyword in keywords):
                detected_emotions.append(emotion)

        return detected_emotions[0] if detected_emotions else 'neutral'

    def generate_poetic_response(self, user_message: str) -> str:
        """Generate poetic response using Gemini API with enhanced prompting"""

        # Analyze user's emotional state
        emotion = self.analyze_user_emotion(user_message)

        # Build context-aware prompt
        enhanced_prompt = f"""
        {create_poetic_system_prompt()}

        CURRENT CONTEXT:
        - User's detected emotion: {emotion}
        - User's message: "{user_message}"
        - Conversation history length: {len(self.conversation_history)}

        SPECIAL INSTRUCTIONS FOR THIS RESPONSE:
        - Acknowledge their {emotion} emotional state poetically
        - Use metaphors that resonate with {emotion} feelings
        - Create a response that offers comfort, celebration, or reflection as appropriate
        - Keep the response between 2-4 lines unless the message calls for something longer

        Now, transform their message into a beautiful poetic response that speaks to their soul.
        """

        try:
            # Generate response using Gemini
            response = model.generate_content(enhanced_prompt)
            poem = response.text.strip()

            # Store in conversation history
            self.conversation_history.append({
                'user': user_message,
                'bot': poem,
                'emotion': emotion,
                'timestamp': time.time()
            })

            return poem

        except Exception as e:
            # Fallback poetic response if API fails
            return f"In digital whispers, words may fail,\nBut poetry's spirit will always prevail.\nLet us try again, dear soul so bright,\nTo weave your thoughts in lyrical light."

    def start_conversation(self):
        """Main conversation loop with beautiful interface"""

        # Display welcome message
        welcome_html = '''
        <div style="
            text-align: center;
            background: linear-gradient(45deg, #667eea, #764ba2);
            color: white;
            padding: 40px;
            border-radius: 20px;
            margin: 20px 0;
            box-shadow: 0 15px 35px rgba(0,0,0,0.3);
        ">
            <h1 style="font-family: 'Georgia', serif; margin-bottom: 20px;">🌙 Welcome to Luna's Poetic Realm 🌙</h1>
            <p style="font-size: 18px; font-style: italic; margin-bottom: 30px;">
                "Where every word becomes a verse, and every thought transforms into poetry..."
            </p>
            <div style="
                background: rgba(255,255,255,0.2);
                padding: 20px;
                border-radius: 15px;
                margin: 20px auto;
                max-width: 600px;
                backdrop-filter: blur(10px);
            ">
                <h3>How to Chat with Luna:</h3>
                <p>• Share your feelings, thoughts, or experiences</p>
                <p>• Ask questions about life, love, or dreams</p>
                <p>• Tell me about your day or current mood</p>
                <p>• Type 'goodbye' when you wish to leave</p>
            </div>
        </div>
        '''
        display(HTML(welcome_html))

        # Initial poetic greeting
        greeting_poem = """Greetings, wanderer of words and dreams,
I am Luna, where starlight gleams.
Share your heart, both joy and strife,
And I'll weave poetry from your life."""

        self.typing_effect.display_with_typing_effect(greeting_poem, 'contemplative')

        print("\n" + "="*60)
        print("✨ Start typing your message below ✨")
        print("="*60)

In [8]:
# =============================================================================
# INTERACTIVE CHAT FUNCTION
# =============================================================================

def run_poetic_chat():
    """Function to run the interactive chat in Google Colab"""

    # Check if API key is set
    if API_KEY == "YOUR_GEMINI_API_KEY_HERE":
        print("🚨 Please set your Gemini API key in the API_KEY variable above!")
        print("Get your key from: https://makersuite.google.com/app/apikey")
        return

    # Initialize chatbot
    chatbot = PoeticChatbot()
    chatbot.start_conversation()

    # Chat loop
    while True:
        try:
            # Get user input
            user_input = input("\n💭 You: ").strip()

            # Check for exit conditions
            if user_input.lower() in ['goodbye', 'bye', 'exit', 'quit']:
                farewell_poem = """Farewell, dear soul, until we meet again,
In realms where poetry shall ever reign.
May your days be filled with verses bright,
And dreams that dance in starry night."""

                chatbot.typing_effect.display_with_typing_effect(farewell_poem, 'slow')
                print("\n🌙 Thank you for sharing your soul with Luna! 🌙")
                break

            # Skip empty messages
            if not user_input:
                continue

            # Generate and display poetic response
            poem = chatbot.generate_poetic_response(user_input)

            # Determine typing speed based on poem length
            if len(poem) > 200:
                speed = 'fast'
            elif len(poem) > 100:
                speed = 'normal'
            else:
                speed = 'contemplative'

            chatbot.typing_effect.display_with_typing_effect(poem, speed)

        except KeyboardInterrupt:
            print("\n\n🌙 Until next time, poetry lover! 🌙")
            break
        except Exception as e:
            print(f"🌙 Luna whispers: Something mystical happened... {str(e)}")
            continue

In [9]:
# =============================================================================
# TESTING AND DEMONSTRATION
# =============================================================================

def demo_poetic_responses():
    """Demonstrate the chatbot with sample inputs"""

    sample_messages = [
        "I'm feeling really happy today!",
        "I had a tough day at work and I'm exhausted",
        "I'm confused about my future career path",
        "The sunset was absolutely beautiful tonight",
        "I miss my grandmother who passed away last year"
    ]

    chatbot = PoeticChatbot()

    print("🌟 DEMO: Sample Poetic Responses 🌟")
    print("="*50)

    for i, message in enumerate(sample_messages, 1):
        print(f"\n{i}. User Input: '{message}'")
        print("-" * 40)

        poem = chatbot.generate_poetic_response(message)
        print("Luna's Response:")
        print(poem)
        print("-" * 40)
        time.sleep(2)  # Pause between examples

In [10]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("🌙✨ Poetic Personality Chatbot Ready! ✨🌙")
    print("\nChoose an option:")
    print("1. Start Interactive Chat")
    print("2. Run Demo with Sample Responses")
    print("3. View System Prompt")

    choice = input("\nEnter your choice (1-3): ").strip()

    if choice == "1":
        run_poetic_chat()
    elif choice == "2":
        demo_poetic_responses()
    elif choice == "3":
        print("\n" + "="*60)
        print("SYSTEM PROMPT FOR POETIC PERSONALITY:")
        print("="*60)
        print(create_poetic_system_prompt())
    else:
        print("Starting interactive chat by default...")
        run_poetic_chat()


🌙 Thank you for sharing your soul with Luna! 🌙


In [11]:
# =============================================================================
# ADDITIONAL UTILITIES
# =============================================================================

def save_conversation_history(chatbot, filename="poetry_conversation.json"):
    """Save conversation history to JSON file"""
    with open(filename, 'w') as f:
        json.dump(chatbot.conversation_history, f, indent=2)
    print(f"Conversation saved to {filename}")

def analyze_poetry_patterns(chatbot):
    """Analyze patterns in generated poetry"""
    if not chatbot.conversation_history:
        print("No conversation history to analyze.")
        return

    emotions = [entry['emotion'] for entry in chatbot.conversation_history]
    emotion_counts = {}
    for emotion in emotions:
        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

    print("\n📊 Conversation Analysis:")
    print("="*30)
    print("Emotions Detected:")
    for emotion, count in emotion_counts.items():
        print(f"  {emotion.title()}: {count} messages")

    avg_response_length = sum(len(entry['bot']) for entry in chatbot.conversation_history) / len(chatbot.conversation_history)
    print(f"\nAverage Response Length: {avg_response_length:.1f} characters")